# Livrable 1 - groupe 1

Nous sommes le groupe 1 composé de :  
- DELORME Alexandre
- ENCOGNERE Yanis
- MENNERON Laurine
- PEREON Alexandre
- ROCHARD Léo

## Contenu du livrable
TODO...


# TODO

##  Analyse des Résultats et Biais/VarianceAnalyse

Discussion sur le surapprentissage/sous-apprentissage

Calcul et affichage d'une matrice de confusion

Calcul de métriques supplémentaires (précision, rappel, F1-score)

## À implémenter/discuter:

Régularisation L2 dans les couches denses/convolutionnelles

Ajustement du taux de dropout

Utilisation de Batch Normalization

Transfer learning avec un modèle pré-entraîné (VGG16, ResNet, etc.)

Grid search pour optimiser les hyperparamètres

## Documentation et Justifications

Schéma de l'architecture du réseau (peut être généré avec tf.keras.utils.plot_model)

Justification des choix (architecture, hyperparamètres, etc.)

Analyse des résultats obtenus

Discussion sur les difficultés rencontrées (images réalistes dans les peintures)

## Gestion des Données Déséquilibrées

Analyse du déséquilibre entre classes photo/non-photo

Techniques pour gérer le déséquilibre (poids de classe, oversampling, etc.)

## EDA
TODO...
- Chargement du dataset
- Visualisation
- Répartition des données

### Chargement des bibliothèques

In [ ]:
%load_ext tensorboard
import imghdr
import os
import pathlib
import zipfile
from datetime import datetime

import gdown
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

### Chargement du dataset


In [ ]:
# ID du fichier (extrait de l'URL)
file_id = "1Q5QttYirEHvto38l6e6uk66Fjvuk_V7I"
dataset_path = "dataset_livrable_1"
zip_path = dataset_path + ".zip"
extract_dir = pathlib.Path(zip_path).parent / dataset_path

if not os.path.exists(extract_dir):
    print(f"Le dossier '{extract_dir}' n'existe pas. Téléchargement en cours...")
    gdown.download(f"https://drive.google.com/uc?id={file_id}", zip_path, quiet=False)

    print(f"Extraction ZIP en cours...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extraction Zip terminée")
else:
    print(f"Le dossier '{extract_dir}' existe déjà. Téléchargement et extraction non nécessaires.")

data_dir = extract_dir
print(f"Dataset disponible dans : {data_dir}")

In [ ]:
categories = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
print(f"Catégories détectées : {categories}")

### Pré traitement des images

In [ ]:
# Function to detect and remove corrupted files
num_skipped = 0
for folder_name in categories:
    folder_path = data_dir / folder_name
    for fname in os.listdir(folder_path):
        fpath = folder_path / fname
        try:
            # Check if the file is a valid image
            if imghdr.what(fpath) is None:
                raise IOError(f"File is not a valid image: {fpath}")

            # Try to open the image file
            with tf.io.gfile.GFile(fpath, 'rb') as f:
                img_content = f.read()
            try:
                img = tf.io.decode_image(img_content, dtype=tf.dtypes.uint8, channels=3)
            except tf.errors.InvalidArgumentError as e:
                print(f"Invalid image detected and removed: {fpath}")
                num_skipped += 1
                os.remove(fpath)
                continue
        except (IOError, SyntaxError) as e:
            print(f"Corrupted file detected and removed: {fpath}")
            num_skipped += 1
            os.remove(fpath)

print(f"Number of corrupted files removed: {num_skipped}")

### Chargement des images

In [ ]:
validation_split = 0.2
seed = 42
batch_size = 32
img_height = 180
img_width = 180

train_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    class_names=categories)

val_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    class_names=categories)

class_names = train_set.class_names
num_classes = len(class_names)

print(f"Classes found: {class_names}")

### Visualisation des différentes classes

In [ ]:
plt.figure(figsize=(8, 8))
for images, labels in train_set.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

### Répartition des classes au sein du dataset

In [ ]:
def count_labels(dataset):
    label_counts = {}
    for _, labels in dataset:
        for label in labels.numpy():
            class_name = class_names[label]
            label_counts[class_name] = label_counts.get(class_name, 0) + 1
    return label_counts


# Compter les labels dans les jeux d'entraînement et de validation
train_label_counts = count_labels(train_set)
val_label_counts = count_labels(val_set)

# Extraire les données pour la visualisation
labels, train_counts = zip(*train_label_counts.items())

# Afficher les résultats
print("Répartition des labels dans le jeu d'entraînement :")
print(train_label_counts)

# Affichage de la répartition
plt.figure(figsize=(8, 4))
plt.bar(labels, train_counts, color="skyblue")
plt.xticks(rotation=45)
plt.title('Répartition des labels - Jeu d\'entraînement')
plt.xlabel('Labels')
plt.ylabel('Nombre d\'images')
plt.tight_layout()
plt.show()

## Modélisation

### Optimisation pour l'entrainement

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Utiliser que 1/10 des données pour l'entrainement
train_size = int(len(train_set) // 10)
train_set_small = train_set.take(train_size)
train_set_small = train_set_small.shuffle(1000).prefetch(buffer_size=AUTOTUNE)

# Valider avec votre jeu de validation
val_set = val_set.prefetch(buffer_size=AUTOTUNE)

### Définition du modèle

In [ ]:
model = tf.keras.Sequential(layers=[
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.1, seed=42),

    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.1, seed=42),

    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.1, seed=42),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(0.3, seed=42),
    tf.keras.layers.Dense(units=num_classes, activation='softmax'), ]
)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

model.summary()

### Entrainement

#### Définition des callbacks

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='./L1_model.keras', save_best_only=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#### Fit

In [ ]:
epochs = 20

history = model.fit(train_set,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=val_set,
                    verbose=1,
                    callbacks=[early_stopping, model_checkpoint])

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
%tensorboard --logdir logs/fit

### Matrice de confusion

In [ ]:
val_predictions = model.predict(val_set)
val_pred_labels = np.argmax(val_predictions, axis=1)

val_true_labels = np.concatenate([y for x, y in val_set], axis=0)

conf_matrix = confusion_matrix(val_true_labels, val_pred_labels)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title("Matrice de Confusion")
plt.xlabel("Classe Prédite")
plt.ylabel("Classe Réelle")
plt.show()